In [1]:
from pyomo.environ import AbstractModel, Set, Param, Var, Constraint, ConstraintList, Objective, NonNegativeIntegers, \
    NonNegativeReals, Reals, maximize
import os, re
os.environ['path'] = os.pathsep.join(['K:\\COIN-OR\\1.8.0\\win32-msvc12\\bin', 'C:\\Users\\watso\\coin-or\\bin', os.environ['path']])

In [612]:
target_cals = 2000
target_protein = 90*1.6 / target_cals * 4 * 100
#target_macros = {'fat': 30, 'carbs': 25, 'protein': 45}  # fat loss
#target_macros = {'fat': 15, 'carbs': 55, 'protein': 30}  # muscle gain
target_macros = {'fat': 15, 'carbs': 100 - 15 - target_protein, 'protein': target_protein}  # muscle gain
#target_macros = {'fat': 62.5, 'carbs': 5, 'protein': 32.5}  # keto
targets = {'fat': target_macros['fat'] / 9 * target_cals / 100,
           'carbs': target_macros['carbs'] / 4 * target_cals / 100,
           'protein': target_macros['protein'] / 4 * target_cals / 100}
print('targets =', targets)
amounts = {
    'egg': 3,
    'heritage flakes': (int(0.8*53),53),
    'skim milk': (int(0.8*240),240),
    'orange juice': (int(0.5*240),240),
    
    #'supergrains': (0,1),
    #'raisins': (0,1),

    'smoothie': (0,1),
    'whey protein': (0,26),

    'lettuce': 15,
    'tomatoes': 35,
    'pickle': 28,
    'mustard': 5,
    'ezekiel bread': 2,
    'low sodium deli turkey': (44,65),
    
    'apple': (0,1),

    '0% fruit yogurt': (0,2),
    #'2% fruit yogurt': (0,2),
    #'fage 0% yogurt': (0,170),
    #'fage 2% yogurt': (0,170),
    #'chobani strawberry': (0,340),
    
    #'pure protein bar': (0,1),

    'chicken': (4,7),
    'broccoli': 72,
    'rice': (0,1),
    'salad': 1,
    'lite creamy caesar': (20,30),
    'honey': 1,
}

targets = {'fat': 33.333333333333336, 'carbs': 281.0, 'protein': 143.99999999999997}


In [613]:
from collections import namedtuple
Food = namedtuple('Food', ['fat', 'carbs', 'protein', 'sodium', 'potassium', 'fiber', 'units'])
food = {
    'egg': Food(fat=5.01, carbs=0.483, protein=6.24, sodium=65, potassium=66, fiber=0, units='large'),
    'chicken': Food(fat=0.998, carbs=0, protein=8.57, sodium=102, potassium=102, fiber=0, units='ounces'), 
    'broccoli': Food(fat=0.0017, carbs=0.0454, protein=0.0235, sodium=1.3, potassium=1.4, fiber=0.028, units='grams'),  # 1/2 cup = 78 g
    'lettuce': Food(fat=0.3/85, carbs=2.8/85, protein=1.1/85, sodium=6.8/85, potassium=210/85, fiber=1.78/85, units='grams'),  # 1 serving = 85 g
    'tomatoes': Food(fat=0.002, carbs=0.0389, protein=0.0088, sodium=0.05, potassium=2.37, fiber=0.012, units='grams'),
    'smoothie': Food(fat=0.9743, carbs=52.3659, protein=3.7214, sodium=38.31, potassium=931.88, fiber=6.626, units='serving'),
    'apple': Food(fat=0.34, carbs=27.6, protein=0.52, sodium=2, potassium=214, fiber=4.8, units='medium'),
    'bbq sauce': Food(fat=0, carbs=7/33, protein=0, sodium=250/33, potassium=0, fiber=0, units='grams'),
    'mustard': Food(fat=0.5/5, carbs=1/5, protein=0, sodium=100/5, potassium=0, fiber=0, units='grams'),
    'heritage flakes': Food(fat=1.5/40, carbs=31/40, protein=5/40, sodium=170/40, potassium=197/40, fiber=7/40, units='grams'),
    'arnold bread': Food(fat=2, carbs=20, protein=5, sodium=160, potassium=90, fiber=3, units='slice'),
    'deli turkey': Food(fat=1/56, carbs=1/56, protein=13/56, sodium=420/56, potassium=0, fiber=0, units='grams'),
    'low sodium deli turkey': Food(fat=1/56, carbs=0/56, protein=13/56, sodium=210/56, potassium=0, fiber=0, units='grams'),
    '0% fruit yogurt': Food(fat=0, carbs=17, protein=11, sodium=60, potassium=190, fiber=0.5, units='container'),
    '2% fruit yogurt': Food(fat=2.5, carbs=16, protein=11, sodium=65, potassium=200, fiber=0.5, units='container'),
    'fage 0% yogurt': Food(fat=0, carbs=5/170, protein=18/170, sodium=65/170, potassium=260/170, fiber=0, units='grams'),
    'fage 2% yogurt': Food(fat=4/200, carbs=6/200, protein=20/200, sodium=65/200, potassium=0/200, fiber=0, units='grams'),
    'supergrains': Food(fat=4.5, carbs=38, protein=7, sodium=180, potassium=180, fiber=5, units='packets'),
    'pickle': Food(fat=0, carbs=8/28, protein=0, sodium=150/28, potassium=0, fiber=0, units='grams'),
    'skim milk': Food(fat=0.2/248, carbs=12.3/248, protein=8.4/248, sodium=104.3/248, potassium=387.4/248, fiber=0, units='grams'),
    'whey protein': Food(fat=1/26, carbs=2/26, protein=20/26, sodium=35/26, potassium=132/26, fiber=0/26, units='grams'),
    'pure protein bar': Food(fat=5, carbs=20, protein=19, sodium=210, potassium=45, fiber=0, units='bars'),
    'orange juice': Food(fat=0.5/240, carbs=26.2/240, protein=1.8/240, sodium=2.5/240, potassium=503.2/240, fiber=0.5/240, units='grams'),
    'raisins': Food(fat=0, carbs=11, protein=0.5, sodium=2.5, potassium=110, fiber=1, units='mini box'),
    'rice': Food(fat=2.5, carbs=40, protein=4, sodium=10, potassium=75, fiber=1, units='servings'),
    'salad': Food(fat=0, carbs=3, protein=1, sodium=25, potassium=201, fiber=1, units='servings'),
    'creamy italian': Food(fat=12/29, carbs=2/29, protein=0, sodium=290/29, potassium=0, fiber=0, units='grams'),
    'lite creamy caesar': Food(fat=9/30, carbs=4/30, protein=1/30, sodium=340/30, potassium=0, fiber=1/30, units='grams'),
    'honey': Food(fat=0, carbs=16.5/3, protein=0.06/3, sodium=0.8/3, potassium=10.4/3, fiber=0.04/3, units='tsp'),
    'flax seed': Food(fat=6/14, carbs=3/14, protein=3/14, sodium=0, potassium=61/14, fiber=3/14, units='grams'),
    'chobani strawberry': Food(fat=0, carbs=26/227, protein=18/227, sodium=90/227, potassium=290/227, fiber=0.5/227, units='grams'),
    'ezekiel bread': Food(fat=0.5, carbs=15, protein=5, sodium=75, potassium=81, fiber=3, units='slice'),
}
ints = ['egg', 'cheese stick', 'cheese brick', 'yogurt/maple syrup', 'smoothie', 'apple', 'banana',
        'cottage cheese', 'gummies', 'fruit yogurt', 'bread', 'pickle', 'supergrains', 'olive oil',
        'daddy bar', 'spicy crackers', '0% fruit yogurt', '2% fruit yogurt', 'fruit cup', 'tuna', 'raisins', 'rice', 'salad',
        'honey', 'pure protein bar']
for key, val in food.items():
    food[key]._replace(carbs=val.carbs - val.fiber)
import prettytable
pt = prettytable.PrettyTable(['Name', 'Fat', 'Carbs', 'Protein', 'Sodium', 'Potassium', 'Fiber', 'Units'])
for key, val in sorted(food.items()):
    pt.add_row([key, f'{val.fat:.6g}', f'{val.carbs:.6g}', f'{val.protein:.6g}', f'{val.sodium:.6g}', f'{val.potassium:.6g}',
                f'{val.fiber:.6g}', val.units])
# print(pt)

In [614]:
model = AbstractModel()

In [615]:
model.F = Set(initialize=list(amounts.keys()))
model.N = Set(initialize=['fat', 'carbs', 'protein', 'sodium', 'fiber', 'potassium'])
model.Food = Set(initialize=list(food.keys()))

In [616]:
def a_init(model, f, n):
    return getattr(food[f], n)
model.a = Param(model.Food, model.N, within=NonNegativeReals, initialize=a_init)

In [617]:
model.x = Var(model.F, within=NonNegativeReals)
model.x_fat = Var(within=Reals)
model.x_carbs = Var(within=Reals)
model.x_protein = Var(within=Reals)

In [618]:
def obj_rule(model):
    return 9 * model.x_fat + 4 * model.x_protein + 4 * model.x_carbs
    #return sum(model.x[f] * model.a[f, 'fiber'] for f in model.F)
model.obj = Objective(rule=obj_rule)
#model.obj = Objective(rule=obj_rule, sense=maximize)

In [619]:
def sodium_ub_rule(model):
    return sum(model.x[f] * model.a[f, 'sodium'] for f in model.F) <= 2000
model.sodium_ub_constraint = Constraint(rule=sodium_ub_rule)

In [620]:
def sodium_lb_rule(model):
    return sum(model.x[f] * model.a[f, 'sodium'] for f in model.F) >= 1500
model.sodium_lb_constraint = Constraint(rule=sodium_lb_rule)

In [621]:
def potassium_ub_rule(model):
    return sum(model.x[f] * model.a[f, 'potassium'] for f in model.F) <= 4700
model.potassium_ub_constraint = Constraint(rule=potassium_ub_rule)

In [622]:
def potassium_lb_rule(model):
    return sum(model.x[f] * model.a[f, 'potassium'] for f in model.F) >= 3500
model.potassium_lb_constraint = Constraint(rule=potassium_lb_rule)

In [623]:
def fiber_ub_rule(model, f):
    return sum(model.x[f] * model.a[f, 'fiber'] for f in model.F) <= 38
model.fiber_ub_constraint = Constraint(model.F, rule=fiber_ub_rule)

In [624]:
def fiber_lb_rule(model, f):
    return sum(model.x[f] * model.a[f, 'fiber'] for f in model.F) >= 30
model.fiber_lb_constraint = Constraint(model.F, rule=fiber_lb_rule)

In [625]:
def x_fat_lb_rule(model):
    return sum(model.x[f] * model.a[f, 'fat'] for f in model.F) - model.x_fat <= targets['fat']
model.x_fat_lb_constraint = Constraint(rule=x_fat_lb_rule)

In [626]:
def x_fat_ub_rule(model):
    return sum(model.x[f] * model.a[f, 'fat'] for f in model.F) + model.x_fat >= targets['fat']
model.x_fat_ub_constraint = Constraint(rule=x_fat_ub_rule)

In [627]:
def x_carbs_lb_rule(model):
    return sum(model.x[f] * model.a[f, 'carbs'] for f in model.F) - model.x_carbs <= targets['carbs']
model.x_carbs_lb_constraint = Constraint(rule=x_carbs_lb_rule)

In [628]:
def x_carbs_ub_rule(model):
    return sum(model.x[f] * model.a[f, 'carbs'] for f in model.F) + model.x_carbs >= targets['carbs']
model.x_carbs_ub_constraint = Constraint(rule=x_carbs_ub_rule)

In [629]:
def x_protein_lb_rule(model):
    return sum(model.x[f] * model.a[f, 'protein'] for f in model.F) - model.x_protein <= targets['protein']
model.x_protein_lb_constraint = Constraint(rule=x_protein_lb_rule)

In [630]:
def x_protein_ub_rule(model):
    return sum(model.x[f] * model.a[f, 'protein'] for f in model.F) + model.x_protein >= targets['protein']
model.x_protein_ub_constraint = Constraint(rule=x_protein_ub_rule)

In [631]:
from pyomo.opt import SolverFactory
opt = SolverFactory('cbc')
instance = model.create_instance()
for f in instance.F:
    if isinstance(amounts[f], tuple):
        instance.x[f].setlb(int(amounts[f][0]))
        instance.x[f].setub(int(amounts[f][1]))
        instance.x[f].value = instance.x[f].lb
    else:
        instance.x[f] = amounts[f]
        instance.x[f].fixed = True
    if food[f].units in ['grams', 'ounces'] or f in ints:
        instance.x[f].domain = NonNegativeIntegers
import prettytable
pt_var = prettytable.PrettyTable(['Food', 'Lower', 'Upper', 'Initial'])
pt_fix = prettytable.PrettyTable(['Food', 'Value'])
for f in instance.F:
    if not instance.x[f].fixed:
        pt_var.add_row([f.title(), instance.x[f].lb, instance.x[f].ub, instance.x[f].value])
    else:
        pt_fix.add_row([f.title(), instance.x[f].value])
print(pt_var)
print(pt_fix)
print(f'Integers: {[f for f in instance.F if instance.x[f].domain is NonNegativeIntegers]}')
results = opt.solve(instance)
print(results)

+------------------------+-------+-------+---------+
|          Food          | Lower | Upper | Initial |
+------------------------+-------+-------+---------+
|    Heritage Flakes     |   42  |   53  |    42   |
|       Skim Milk        |  192  |  240  |   192   |
|      Orange Juice      |  120  |  240  |   120   |
|        Smoothie        |   0   |   1   |    0    |
|      Whey Protein      |   0   |   26  |    0    |
| Low Sodium Deli Turkey |   44  |   65  |    44   |
|         Apple          |   0   |   1   |    0    |
|    0% Fruit Yogurt     |   0   |   2   |    0    |
|        Chicken         |   4   |   7   |    4    |
|          Rice          |   0   |   1   |    0    |
|   Lite Creamy Caesar   |   20  |   30  |    20   |
+------------------------+-------+-------+---------+
+---------------+-------+
|      Food     | Value |
+---------------+-------+
|      Egg      |   3   |
|    Lettuce    |   15  |
|    Tomatoes   |   35  |
|     Pickle    |   28  |
|    Mustard    |   5  

In [632]:
def fmt(x):
    if x < 0.1:
        return "0"
    if x < 1:
        return f"{x:.1f}"
    return int(x)

fat = carbs = protein = fiber = sodium = potassium = 0
import prettytable
import textwrap
pt = prettytable.PrettyTable(['Food', 'Value', 'Input', 'Fat', 'Carbs', 'Protein', 'Sodium', 'Potassium', 'Fiber', 'Calories'])
for f in sorted(instance.F):
    value = instance.x[f].value
    fat += value * instance.a[f, 'fat']
    carbs += value * instance.a[f, 'carbs']
    protein += value * instance.a[f, 'protein']
    fiber += value * instance.a[f, 'fiber']
    sodium += value * instance.a[f, 'sodium']
    potassium += value * instance.a[f, 'potassium']
    units = food[f].units
    if f == 'yogurt/maple syrup':
        units = f'{units}, {20*value//170} g maple syrup'
    pt.add_row([f,
                '\n'.join(textwrap.wrap(f'{value} {units}', 20)),
                amounts[f],
                f"{fmt(value * instance.a[f, 'fat'])}",
                f"{fmt(value * instance.a[f, 'carbs'])}",
                f"{fmt(value * instance.a[f, 'protein'])}",
                f"{fmt(value * instance.a[f, 'sodium'])}",
                f"{fmt(value * instance.a[f, 'potassium'])}",
                f"{fmt(value * instance.a[f, 'fiber'])}",
                f"{int(9*value*instance.a[f,'fat']+4*value*instance.a[f,'carbs']+4*value*instance.a[f,'protein'])}",
               ])

cals = 9 * fat + 4 * carbs + 4 * protein

for field in ['Input', 'Fat', 'Carbs', 'Protein', 'Sodium', 'Potassium', 'Fiber', 'Calories']:
    pt.align[field] = "r"

ts = pt.get_string(junction_char='|').splitlines()
print('\n'.join([ts[1]]+[re.sub(r'-(-+)-', r':\1:', ts[2])]+ts[3:-1]))
print()
print(f'Total Calories = {int(cals)}, Fat = {fat:4.1f}, Carbs = {carbs:.1f}, Protein = {protein:5.1f}, Sodium = {int(sodium):4d}, Potassium = {int(potassium):4d}, Fiber = {fiber:4.1f}<br/>')
print(f'   Target Cals = {target_cals}, Fat = {targets["fat"]:.1f}, Carbs = {targets["carbs"]:.1f}, Protein = {targets["protein"]:.1f}, Sodium in [1500,2000], Potassium in [3500,4700], Fiber in [30,38]<br/>')
print(f'Total Macros:  Fat = {fat*9/cals*100:4.1f}%, Carbs = {carbs*4/cals*100:4.1f}%, Protein = {protein*4/cals*100:4.1f}%<br/>')
print(f'     Targets:  Fat = {target_macros["fat"]:4.1f}%, Carbs = {target_macros["carbs"]:4.1f}%, Protein = {target_macros["protein"]:4.1f}%')

|          Food          |     Value     |      Input | Fat | Carbs | Protein | Sodium | Potassium | Fiber | Calories |
|:----------------------:|:-------------:|:----------:|:---:|:-----:|:-------:|:------:|:---------:|:-----:|:--------:|
|    0% fruit yogurt     | 2.0 container |     (0, 2) |   0 |    34 |      22 |    120 |       380 |     1 |      224 |
|         apple          |   1.0 medium  |     (0, 1) | 0.3 |    27 |     0.5 |      2 |       214 |     4 |      115 |
|        broccoli        |    72 grams   |         72 | 0.1 |     3 |       1 |     93 |       100 |     2 |       20 |
|        chicken         |   4.0 ounces  |     (4, 7) |   3 |     0 |      34 |    408 |       408 |     0 |      173 |
|          egg           |    3 large    |          3 |  15 |     1 |      18 |    195 |       198 |     0 |      215 |
|     ezekiel bread      |    2 slice    |          2 |   1 |    30 |      10 |    150 |       162 |     6 |      169 |
|    heritage flakes     |   42.0 grams 

|          Food          |     Value     |      Input | Fat | Carbs | Protein | Sodium | Potassium | Fiber | Calories |
|:----------------------:|:-------------:|:----------:|:---:|:-----:|:-------:|:------:|:---------:|:-----:|:--------:|
|    0% fruit yogurt     | 2.0 container |     (0, 2) |   0 |    34 |      22 |    120 |       380 |     1 |      224 |
|         apple          |   1.0 medium  |     (0, 1) | 0.3 |    27 |     0.5 |      2 |       214 |     4 |      115 |
|        broccoli        |    72 grams   |         72 | 0.1 |     3 |       1 |     93 |       100 |     2 |       20 |
|        chicken         |   4.0 ounces  |     (4, 7) |   3 |     0 |      34 |    408 |       408 |     0 |      173 |
|          egg           |    3 large    |          3 |  15 |     1 |      18 |    195 |       198 |     0 |      215 |
|     ezekiel bread      |    2 slice    |          2 |   1 |    30 |      10 |    150 |       162 |     6 |      169 |
|    heritage flakes     |   42.0 grams  |   (42, 53) |   1 |    32 |       5 |    178 |       206 |     7 |      165 |
|         honey          |     1 tsp     |          1 |   0 |     5 |       0 |    0.3 |         3 |     0 |       22 |
|        lettuce         |    15 grams   |         15 |   0 |   0.5 |     0.2 |      1 |        37 |   0.3 |        3 |
|   lite creamy caesar   |   20.0 grams  |   (20, 30) |   6 |     2 |     0.7 |    226 |         0 |   0.7 |       67 |
| low sodium deli turkey |   44.0 grams  |   (44, 65) | 0.8 |     0 |      10 |    165 |         0 |     0 |       47 |
|        mustard         |    5 grams    |          5 | 0.5 |     1 |       0 |    100 |         0 |     0 |        8 |
|      orange juice      |  223.0 grams  | (120, 240) | 0.5 |    24 |       1 |      2 |       467 |   0.5 |      108 |
|         pickle         |    28 grams   |         28 |   0 |     8 |       0 |    150 |         0 |     0 |       32 |
|          rice          |  1.0 servings |     (0, 1) |   2 |    40 |       4 |     10 |        75 |     1 |      198 |
|         salad          |   1 servings  |          1 |   0 |     3 |       1 |     25 |       201 |     1 |       16 |
|       skim milk        |  230.0 grams  | (192, 240) | 0.2 |    11 |       7 |     96 |       359 |     0 |       78 |
|        smoothie        |  1.0 serving  |     (0, 1) | 1.0 |    52 |       3 |     38 |       931 |     6 |      233 |
|        tomatoes        |    35 grams   |         35 |   0 |     1 |     0.3 |      1 |        82 |   0.4 |        7 |
|      whey protein      |   26.0 grams  |    (0, 26) |   1 |     2 |      20 |     35 |       132 |     0 |       97 |

Total Calories = 2003, Fat = 34.6, Carbs = 281.0, Protein = 142.0, Sodium = 1999, Potassium = 3959, Fiber = 31.7<br/>
   Target Cals = 2000, Fat = 33.3, Carbs = 281.0, Protein = 144.0, Sodium in [1500,2000], Potassium in [3500,4700], Fiber in [30,38]<br/>
Total Macros:  Fat = 15.5%, Carbs = 56.1%, Protein = 28.4%<br/>
     Targets:  Fat = 15.0%, Carbs = 56.2%, Protein = 28.8%

|      Food      |    Value    |   Input    |   Fat   |  Carbs  | Protein |  Sodium |  Fiber   |   Cals  |
|:--------------:|:-----------:|:----------:|:-------:|:-------:|:-------:|:-------:|:--------:|:-------:|
|   bbq sauce    |  26.0 grams |  (20, 30)  |    0    |  4.875  |    0    |   195   |  0.8125  |   19.5  |
|    broccoli    |   78 grams  |     78     |   0.32  |   5.6   |   1.86  |    32   |   2.6    |  32.72  |
|     carrot     |   65 grams  |     65     |  0.156  |  6.227  |  0.6045 |  44.85  |   1.82   |  28.73  |
|     celery     |   65 grams  |     65     |  0.1105 |  1.9305 |  0.4485 |    52   |   1.04   | 10.5105 |
|    chicken     | 11.0 ounces |  (6, 12)   |  11.781 |    0    | 102.366 |  244.2  |    0     | 515.493 |
|    fish oil    |   3 pills   |     3      |   4.5   |   1.5   |   0.75  |    0    |    0     |   49.5  |
|   fruit cup    |  1 package  |     1      |  0.078  |  11.722 |   0.37  |   3.6   |   1.28   |  49.07  |
|    gummies     |  4 gummies  |     4      |    0    |    14   |    0    |    0    |    8     |    56   |
|     hummus     |  15.0 grams |  (15, 30)  | 2.67857 | 2.14286 | 1.07143 | 66.9643 | 0.535714 | 36.9643 |
|    lettuce     |   70 grams  |     70     |  0.105  |  2.009  |  0.952  |   19.6  |   0.91   |  12.789 |
|  maple syrup   |  13.0 grams |  (10, 20)  |  0.0078 |  8.7152 |  0.0052 |   1.56  |    0     | 34.9518 |
| peanut butter  |  20.0 grams |  (15, 30)  | 16.6667 |    4    |    6    | 1.33333 | 1.33333  |   190   |
| ranch dressing |  21.0 grams |  (20, 30)  |   3.5   |   2.8   |   0.7   |   217   |    0     |   45.5  |
|    tomatoes    |  48.0 grams |  (35, 70)  |  0.096  |  1.8672 |  0.4224 |   2.4   |  0.576   | 10.0224 |
|     yogurt     | 192.0 grams | (140, 280) |    0    | 7.61233 | 19.4537 | 71.8943 |    0     | 108.264 |

Total Calories = 1200, Fat = 40.0, Carbs = 75.0, Protein = 135.0, Sodium =  952, Fiber = 18.9<br/>
   Target Cals = 1200, Fat = 40.0, Carbs = 75.0, Protein = 135.0, Sodium < 2000, Fiber = 30.0<br/>
Total Macros:  Fat = 30.0%, Carbs = 25.0%, Protein = 45.0%<br/>
     Targets:  Fat = 30.0%, Carbs = 25.0%, Protein = 45.0%

|      Food      |    Value    | Input |  Fat   | Carbs  | Protein | Sodium | Fiber |   Cals  |
|:--------------:|:-----------:|:-----:|:------:|:------:|:-------:|:------:|:-----:|:-------:|
|     bread      |   2 slice   |   2   |   6    |   38   |    10   |  360   |   6   |   246   |
|   daddy bar    |    1 bars   |   1   |  2.5   |   26   |    2    |   95   |   3   |  134.5  |
|  deli turkey   |   56 grams  |   56  |   1    |   0    |    13   |  360   |   0   |    61   |
|    fish oil    |   3 pills   |   3   |  4.5   |  1.5   |   0.75  |   0    |   0   |   49.5  |
|  fruit yogurt  | 3 container |   3   |   0    |   54   |    36   |  150   |   3   |   360   |
| golean crunch  |   53 grams  |   53  | 3.127  | 37.842 |  8.851  | 98.58  | 7.685 | 214.915 |
|    lettuce     |   35 grams  |   35  | 0.0525 | 1.0045 |  0.476  |  9.8   | 0.455 |  6.3945 |
|      milk      |  244 grams  |  244  | 4.8312 | 11.712 |  8.052  | 114.68 |   0   | 122.537 |
|    mustard     |   15 grams  |   15  |   0    |   0    |    0    |  360   |   0   |    0    |
| protein powder |   30 grams  |   30  |   4    |   12   |    11   |   0    |   12  |   128   |
|    smoothie    |  1 serving  |   1   |  0.92  | 51.22  |   3.28  |   27   |  6.3  |  226.28 |
|    tomatoes    |   20 grams  |   20  |  0.04  | 0.778  |  0.176  |   1    |  0.24 |  4.176  |

Total Calories = 1553, Fat = 27.0, Carbs = 234.1, Protein =  93.6, Sodium = 1576, Fiber = 38.7<br/>
   Target Cals = 1600, Fat = 26.7, Carbs = 220.0, Protein = 120.0, Sodium < 2000, Fiber = 30.0<br/>
Total Macros:  Fat = 15.6%, Carbs = 60.3%, Protein = 24.1%<br/>
     Targets:  Fat = 15.0%, Carbs = 55.0%, Protein = 30.0%

|         Food         |    Value    |  Input   |    Fat     |  Carbs  | Protein |  Sodium |  Fiber  |   Cals  |
|:--------------------:|:-----------:|:--------:|:----------:|:-------:|:-------:|:-------:|:-------:|:-------:|
|      0% yogurt       | 134.0 grams | (0, 170) | 0.00788235 |    0    | 14.1961 | 52.8118 |    0    | 56.8554 |
|        apple         |  1.0 medium |  (0, 1)  |    0.31    |  25.13  |   0.47  |    2    |   4.4   |  105.19 |
|      bbq sauce       |  27.0 grams | (20, 30) |     0      |  5.0625 |    0    |  202.5  | 0.84375 |  20.25  |
|       broccoli       |   78 grams  |    78    |    0.32    |   5.6   |   1.86  |    32   |   2.6   |  32.72  |
|        cheese        |   1 ounces  |    1     |     9      |    1    |    7    |   180   |    0    |   113   |
|       chicken        |  7.0 ounces | (6, 12)  |   7.497    |    0    |  65.142 |  155.4  |    0    | 328.041 |
|         egg          |   3 large   |    3     |    20.1    |   2.94  |  18.27  |   264   |    0    |  265.74 |
|      fruit cup       |  1 package  |    1     |   0.078    |  11.722 |   0.37  |   3.6   |   1.28  |  49.07  |
|     maple syrup      |  17.0 grams | (0, 20)  |   0.0102   | 11.3968 |  0.0068 |   2.04  |    0    | 45.7062 |
| nofat cottage cheese |    1 cups   |    1     |     0      |    10   |    26   |   820   |    0    |   144   |
|      olive oil       |    1 tbsp   |    1     |     14     |    0    |    0    |    0    |    0    |   126   |
|       spinach        |   60 grams  |    60    |   0.234    |  2.178  |  1.716  |   47.4  |   1.32  |  17.682 |

Total Calories = 1304, Fat = 51.6, Carbs = 75.0, Protein = 135.0, Sodium = 1761, Fiber = 10.4<br/>
   Target Cals = 1200, Fat = 40.0, Carbs = 75.0, Protein = 135.0, Sodium < 2000, Fiber = 30.0<br/>
Total Macros:  Fat = 35.6%, Carbs = 23.0%, Protein = 41.4%<br/>
     Targets:  Fat = 30.0%, Carbs = 25.0%, Protein = 45.0%
